In [12]:
import pandas as pd

imdb = pd.read_table("./data/imdb/title_basics.tsv")

C:\Users\kylei\AppData\Local\Temp\ipykernel_34592\4123186128.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb = pd.read_table("./data/imdb/title_basics.tsv")


In [13]:
non_valid_types = ["tvShort", "tvSeries", "tvEpisode", "tvMiniSeries", "tvSpecial", "videogame", "tvPilot", "short"]

imdb_movies = imdb.loc[imdb["titleType"] != "tvEpisode"]
for item_type in non_valid_types:
    imdb_movies = imdb_movies.loc[imdb_movies["titleType"] != item_type]

imdb_movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama
...,...,...,...,...,...,...,...,...,...
9717404,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,\N,100,Documentary
9717409,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015,\N,66,Drama
9717416,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,\N,\N,Comedy
9717426,tt9916730,movie,6 Gunn,6 Gunn,0,2017,\N,116,\N


In [18]:
import glob
from collections import Counter

all_count = Counter()
for path in glob.iglob("results/*/*/*/*/unmatched_titles.txt"):
    with open(path, "r") as f:
        for line in f:
            all_count.update({line.strip("\n"): 1})

In [20]:
all_count

Counter({'Princess Mononoke (1997)': 67,
         'Life is Beautiful (1997)': 46,
         'Show Me Love (1998)': 33,
         'Priceless (2006)': 31,
         'Blue Is the Warmest Color (2013)': 30,
         'Bruno (2009)': 30,
         'The Avengers (2012)': 30,
         'Once Upon a Time in the West (1968)': 29,
         'Hero (2002)': 28,
         'Mad Max: Fury Road (2015)': 27,
         "Howl's Moving Castle (2004)": 26,
         '500 Days of Summer (2009)': 26,
         'Like Water for Chocolate (1992)': 23,
         'In the Mood for Love (2000)': 22,
         'The Tin Drum (1979)': 22,
         'Nosferatu (1922)': 22,
         'District 9 (2009)': 22,
         'The Italian Job (1969)': 22,
         "National Lampoon's Vacation (1983)": 21,
         'Rififi (1955)': 21,
         'Talk to Her (2002)': 19,
         'The Triplets of Belleville (2003)': 19,
         'Up (2009)': 19,
         'A Very Long Engagement (2004)': 19,
         'Ghost in the Shell 2: Innocence (2004)': 19,


In [21]:
imdb_movies["titleYear"] = imdb_movies["primaryTitle"] + " (" +imdb_movies["startYear"] + ")"
imdb_movies = imdb_movies.fillna("")

In [22]:
from src.dictionary import norm_lev_sim
from collections import defaultdict

unmatched_titles = list(all_count.keys())
unmatched_dict = {}

imdb_titles = imdb_movies["titleYear"].values
imdb_title_lookup = {title: None for title in imdb_titles}

In [23]:
def extract_title_year(string: str):
    title, year = string.split(" (")
    year = year.strip(")")
    return title, year


for unmatched_title in unmatched_titles:
    try:
        imdb_title_lookup[unmatched_title]
        matched = imdb_movies.loc[imdb_movies["titleYear"] == unmatched_title]
        yearless_title, _ = extract_title_year(unmatched_title)

        unmatched_dict[yearless_title] = {
            "ReleaseYear": matched["startYear"].values[0],
            "Genres": matched["genres"].values[0],
        }

    except KeyError as e:
        for imdb_title in imdb_titles:
            sim = norm_lev_sim(unmatched_title, imdb_title, None)
            if sim > 0.8:
                matched = imdb_movies.loc[imdb_movies["titleYear"] == imdb_title]
                yearless_title, _ = extract_title_year(unmatched_title)

                unmatched_dict[yearless_title] = {
                    "ReleaseYear": matched["startYear"].values[0],
                    "Genres": matched["genres"].values[0],
                }

In [26]:
unmatched_df = pd.DataFrame(unmatched_dict).T.reset_index().rename({"index": "MovieTitle"}, axis=1)
unmatched_df.to_csv("./data/auxiliary_dataset.csv", index=False)